# Deer Agent Creation
The goal of this notebook is to create an input file for the agent based simulation. 

It will take in some input probabilities (portion of population that is male, adult etc) and create a table that can be used for agent creation. 

In [2]:
import numpy as np
import pandas as pd
import repast4py
import logging as pylog # Repast logger is called as "logging"

from typing import Dict, Tuple
from mpi4py import MPI
from dataclasses import dataclass, field, fields, asdict

import collections
import csv
import os

import random as rndm
import torch
import yaml

from repast4py import core, random, space, schedule, logging, parameters
from repast4py import context 
from repast4py.space import ContinuousPoint as cpt
from repast4py.space import BorderType, OccupancyType
from repast4py.space import DiscretePoint as dpt
from repast4py.value_layer import SharedValueLayer
from datetime import datetime, timedelta

# # Local Files
# from deer_agent.deer_agent import Deer, Deer_Config
# from landscape import fetch_img 

In [5]:
config_file = '../config/local_deer_config.yaml'

## Create Params
Based off of [this](https://www.sciencedirect.com/science/article/pii/S0304380022002162) and the associated appendix.

This is going to create a number of deer agents with their starting params. The repast simulation will take a subset of these deer. 

Static params created for each agent:

    is_infected: bool = False
    is_contagious: bool = False
    has_recovered: bool = False
    disease_timer: int = 0
    random_seed: int = 0
    group_id: int = 0
    birth_Date: datetime = datetime.datetime(2020, 1, 1)
    is_male: bool = False
    is_fawn: bool = False
    gestation_days: int = 0
    has_homerange: bool = False # Assume it's false and check that initial pos is valid for centroid
    current_x: float = 0.0
    current_y: float = 0.0
    behaviour_state: Behaviour_State = field(default = Behaviour_State.NORMAL) 
    
    AND

    last_point = Point(self.current_x,self.current_y)
    current_point = Point(self.current_x,self.current_y)
    centroid = Point(self.current_x,self.current_y)
    self.pos = Position_Vector(last_point, current_point, centroid) 

Some other params that might get calculated:
* $\rho_t$: Center angel of distribution [radians]
* $\mu_t$: Mean Cosine of deviations/ shape parameter [radians?]. 
* $\beta_t$: Scale parameter
* $\alpha_t$: Shape parameter
* FOCUS params:
  * $\theta_{Ct}$
  * $\rho_\infty$
  * $\rho_0$
  * $\gamma_\rho$

In [64]:
dirname = os.getcwd()
filename = os.path.join(dirname, config_file)

with open(filename, 'r') as stream:
    params = yaml.safe_load(stream)


Create dataframe using vectorizable function:
https://stackoverflow.com/questions/61823039/how-to-create-pandas-dataframe-and-fill-it-from-function

EPSG:5070:

x_min = -8587456
x_max = -8555705
y_min = 4729698
y_max = 4751385


![IMAGE](https://www.mrlc.gov/geoserver/ows?service=WCS&version=2.0.1&&REQUEST=GetCoverage&coverageId=mrlc_download:NLCD_2019_Land_Cover_L48&SUBSETTINGCRS=EPSG:5070&subset=X(1603849,1637294)&subset=Y(1950179,1973346)&OUTPUTCRS=EPSG:5070&FORMAT=image/png)

In [137]:
os.getcwd()

'/home/jovyan/work/GMU/Repast4pyModels/Notebooks'

In [31]:
from owslib.wcs import WebCoverageService
import rasterio
from rasterio.plot import reshape_as_raster, reshape_as_image

@dataclass
class WCS_Info:
    '''
    Simple class to hold info required to get image using Web Coverage Service 
    '''
    layer_id: str
    wcs_url: str 
    epsg: str
    description: str
    path: str
    bounds: [] #xmin, ymin, xmax, ymax
    file_format: str = 'GeoTIFF'
    wcs_version: str = '2.0.1'
    x_label: str = 'X'
    y_label: str = 'Y'

In [12]:
############################################################################
## Some common rasters to use
############################################################################
land_cover = WCS_Info(layer_id='mrlc_download__NLCD_2019_Land_Cover_L48',
                     wcs_url = 'https://www.mrlc.gov/geoserver/ows', 
                     epsg = 'EPSG:5070',
                     path = '../input/images/2019_LandCover.tiff',
                     bounds = [1603849, 1637294, 1950179, 1973346], 
                     description = '2019 NLCD Landcover classification for Howard County'
                     )

imp_surf = WCS_Info(layer_id='mrlc_download__NLCD_2019_Impervious_L48',
                     wcs_url = 'https://www.mrlc.gov/geoserver/ows', 
                     epsg = 'EPSG:5070',
                     path = '../input/images/2019_ImpervSurf.tiff',
                     bounds = [1603849, 1637294, 1950179, 1973346], 
                     description = '2019 NLCD Impervious Surfaces for Howard County'
                     ) 

# https://www.mrlc.gov/data/nlcd-2019-usfs-tree-canopy-cover-conus
# https://www.mrlc.gov/geoserver/ows?service=WCS&version=2.0.1&
# &REQUEST=GetCoverage&coverageId=mrlc_download:nlcd_tcc_conus_2019_v2021-4
# &SUBSETTINGCRS=EPSG:4326&subset=Long(-77.187113,-76.696774)&subset=Lat(39.103142,39.369323)&FORMAT=image/tiff

canopy = WCS_Info(layer_id='mrlc_download:nlcd_tcc_conus_2019_v2021-4',
                     wcs_url = 'https://www.mrlc.gov/geoserver/ows', 
                     epsg = 'EPSG:5070',
                     path = '../input/images/2019_CONUS_Canopy.tiff',
                     bounds = [1603849, 1637294, 1950179, 1973346], 
                     description = '2019 NLCD Canopy Estimate for Howard County'
                     )
############################################################################

def fetch_img(WCS_Info):
    '''
    Download WCS image, save it to a file, and return a numpy array of the raster.
    Also return the XY pixel resolution in the projection units.
    '''
    wcs_service = WebCoverageService(WCS_Info.wcs_url,
                            version=WCS_Info.wcs_version)
    img = wcs_service.getCoverage(identifier = [WCS_Info.layer_id],
                                  srs = WCS_Info.epsg,
                                  subsets = [(WCS_Info.x_label, WCS_Info.bounds[0], WCS_Info.bounds[1]),
                                             (WCS_Info.y_label, WCS_Info.bounds[2], WCS_Info.bounds[3])],
                                  format = WCS_Info.file_format)
    
    out = open(WCS_Info.path, 'wb')
    out.write(img.read())
    out.close()

    with rasterio.open(WCS_Info.path, 'r') as ds:
        xy_resolution = ds.res
        arr = reshape_as_image(ds.read())  # read all raster values and return them as (x,y,z) 

    return arr, xy_resolution

In [14]:
canopy_array, xy_resolution = fetch_img(canopy)
canopy_array, xy_resolution = fetch_img(land_cover)
canopy_array, xy_resolution = fetch_img(imp_surf)

In [96]:
! pip install shapely

In [94]:
import geojson 
import shapely
import geopandas as gpd
from rasterio.mask import mask

'{"type":"Polygon","coordinates":[[[1630000.0,1960000.0],[1630000.0,1970000.0],[1615000.0,1970000.0],[1615000.0,1960000.0],[1630000.0,1960000.0]]]}'

In [116]:
bbox = shapely.geometry.box(int(params['geo']['x_min']), 
           int(params['geo']['y_min']), 
           int(params['geo']['x_max']),
           int(params['geo']['y_max']))
geojson_dict =  dict(geojson.loads(shapely.to_geojson(bbox)))

In [119]:
geojson_dict.get('type')

'Polygon'

In [133]:
test_canopy = WCS_Info(layer_id='mrlc_download:nlcd_tcc_conus_2019_v2021-4',
                 wcs_url = 'https://www.mrlc.gov/geoserver/ows', 
                 epsg = 'EPSG:5070',
                 path = params['geo']['tiff_path'],
                 bounds = [int(params['geo']['x_min']), 
                           int(params['geo']['x_max']), 
                           int(params['geo']['y_min']), 
                           int(params['geo']['y_max'])], 
                 description = '2019 NLCD Canopy Estimate for Howard County'
                 )

if os.path.isfile(test_canopy.path): 
    with rasterio.open(test_canopy.path, 'r') as ds:
        xy_resolution = ds.res
        print(xy_resolution)
        image_bounds = ds.bounds
        print(image_bounds)
        print(test_canopy.bounds)
        # arr = reshape_as_image(ds.read(window = window))   # read all raster values and return them as (x,y,z) 
        # x = ds.read() 
        x =reshape_as_image(mask(ds, shapes = [bbox], crop = True)[0])
x

(30.0, 30.0)
BoundingBox(left=1603845.0, bottom=1950165.0, right=1637295.0, top=1973355.0)
[1615000, 1630000, 1960000, 1970000]


array([[[ 0],
        [ 0],
        [ 0],
        ...,
        [ 0],
        [ 0],
        [ 0]],

       [[ 0],
        [ 0],
        [ 0],
        ...,
        [94],
        [82],
        [85]],

       [[ 0],
        [ 0],
        [ 0],
        ...,
        [94],
        [83],
        [94]],

       ...,

       [[ 0],
        [94],
        [89],
        ...,
        [14],
        [15],
        [18]],

       [[ 0],
        [89],
        [90],
        ...,
        [14],
        [20],
        [23]],

       [[ 0],
        [ 0],
        [ 0],
        ...,
        [ 0],
        [ 0],
        [ 0]]], dtype=uint8)

In [129]:
x

(array([[[ 0,  0,  0, ...,  0,  0,  0],
         [ 0,  0,  0, ..., 94, 82, 85],
         [ 0,  0,  0, ..., 94, 83, 94],
         ...,
         [ 0, 94, 89, ..., 14, 15, 18],
         [ 0, 89, 90, ..., 14, 20, 23],
         [ 0,  0,  0, ...,  0,  0,  0]]], dtype=uint8),
 Affine(30.0, 0.0, 1614975.0,
        0.0, -30.0, 1970025.0))

In [ ]:
comm = MPI.COMM_WORLD
  
# Convert projection units to pixel units:
# Array is in (y,x,z)
projection_bounds = space.BoundingBox(0, 
                            canopy_array.shape[1], 
                            0, 
                            canopy_array.shape[0], 
                            0, 
                            0) 
 
canopy_layer = SharedValueLayer(comm = comm, 
                                bounds = projection_bounds, 
                                borders = space.BorderType.Sticky, 
                                buffer_size = int(params['geo']['buffer_size']), 
                                init_value = 0)

sub_array = canopy_array.T[0,0]

canopy_layer.grid = torch.from_numpy(canopy_array).type(torch.float64)

# grid = space.SharedGrid('grid', 
#                         bounds = projection_bounds, 
#                         borders = space.BorderType.Sticky, 
#                         occupancy = OccupancyType.Multiple,
#                         buffer_size = int(params['geo']['buffer_size']), 
#                         comm=comm)

# shared_space = space.SharedCSpace('space', 
#                            bounds = projection_bounds, 
#                            borders = BorderType.Sticky, 
#                            occupancy = OccupancyType.Multiple,
#                            buffer_size = int(params['geo']['buffer_size']), 
#                            comm=comm, 
#                            tree_threshold=100)

# context.add_projection(grid) 
# context.add_projection(shared_space)
# context.add_value_layer(canopy_layer)

In [ ]:
sub_array.shape[1:]

In [ ]:
xy = [canopy_layer.bounds.xmin, 
                    canopy_layer.bounds.xmin + canopy_layer.bounds.xextent,
                    canopy_layer.bounds.ymin, 
                    canopy_layer.bounds.ymin + canopy_layer.bounds.yextent,
                   ]
# xy = [15, 33, 0, 33]

In [ ]:
sub_array = canopy_array[0,xy[0]:xy[1],xy[2]:xy[3]]
canopy_layer.grid[xy[0]:xy[1],xy[2]:xy[3]] = torch.from_numpy(sub_array).type(torch.float64)
sub_array.shape

In [ ]:
canopy_layer.bounds

In [ ]:
canopy_layer.grid.shape

In [ ]:
torch.from_numpy(sub_array)[0,:,:].shape

In [ ]:
canopy_layer.grid.shape

In [ ]:
 1:67

## Now to read the tiff
although it might be easier to just use the memory object, but this allows caching too and eyeballin'


In [ ]:

raster =  rasterio.open('raster.tif')
gt = raster.affine
print gt
Affine(2.11668210080698, 0.0, 258012.37107330866,
   0.0, -2.1168501270110074, 163176.6385398821)
pixelSizeX = gt[0]
pixelSizeY =-gt[4]
print pixelSizeX
2.11668210080698
print pixelSizeY
2.1168501270110074

In [ ]:
import rasterio
with rasterio.open('land_cover2019.tiff', 'r') as ds:
    xy_resolution = ds.res
    arr = ds.read()  # read all raster values

In [ ]:
@dataclass
def Img_Info():
    layer_name
    url
    epsg
    bounds
    format
    description
    

In [ ]:
land_cover = WCS_Info(layer_id='mrlc_download__NLCD_2019_Land_Cover_L48',
                     wcs_url = 'https://www.mrlc.gov/geoserver/ows', 
                     epsg = 'EPSG:5070',
                     path = './tmp/2019_LandCover.tiff',
                     bounds = [1603849, 1637294, 1950179, 1973346], 
                     description = '2019 NLCD Landcover classification for Howard County'
                     )

imp_surf = WCS_Info(layer_id='mrlc_download__NLCD_2019_Impervious_L48',
                     wcs_url = 'https://www.mrlc.gov/geoserver/ows', 
                     epsg = 'EPSG:5070',
                     path = './tmp/2019_ImpervSurf.tiff',
                     bounds = [1603849, 1637294, 1950179, 1973346], 
                     description = '2019 NLCD Impervious Surfaces for Howard County'
                     ) 

# https://www.mrlc.gov/data/nlcd-2019-usfs-tree-canopy-cover-conus
# https://www.mrlc.gov/geoserver/ows?service=WCS&version=2.0.1&
# &REQUEST=GetCoverage&coverageId=mrlc_download:nlcd_tcc_conus_2019_v2021-4
# &SUBSETTINGCRS=EPSG:4326&subset=Long(-77.187113,-76.696774)&subset=Lat(39.103142,39.369323)&FORMAT=image/tiff

canopy = WCS_Info(layer_id='mrlc_download:nlcd_tcc_conus_2019_v2021-4',
                     wcs_url = 'https://www.mrlc.gov/geoserver/ows', 
                     epsg = 'EPSG:5070',
                     path = './tmp/2019_CONUS_Canopy.tiff',
                     bounds = [1603849, 1637294, 1950179, 1973346], 
                     description = '2019 NLCD Canopy Estimate for Howard County'
                     )
############################################################################

'''
Download WCS image, save it to a file, and return a numpy array of the raster.
Also return the XY pixel resolution in the projection units.
'''
wcs_service = WebCoverageService(WCS_Info.wcs_url,
                        version=WCS_Info.wcs_version)
img = wcs_service.getCoverage(identifier = [WCS_Info.layer_id],
                              srs = WCS_Info.epsg,
                              subsets = [(WCS_Info.x_label, WCS_Info.bounds[0], WCS_Info.bounds[1]),
                                         (WCS_Info.y_label, WCS_Info.bounds[2], WCS_Info.bounds[3])],
                              format = WCS_Info.file_format)

out = open(WCS_Info.path, 'wb')
out.write(img.read())
out.close()

with rasterio.open(WCS_Info.path, 'r') as ds:
    xy_resolution = ds.res
    arr = reshape_as_image(ds.read())  # read all raster values and return them as (x,y,z) 
 

In [ ]:
dataset = rasterio.open(canopy.path)

In [ ]:
with rasterio.open(canopy.path, 'r') as ds:
            xy_resolution = ds.res
            image_bounds = ds.bounds
            window = rasterio.Window(WCS_Info.bounds[0], WCS_Info.bounds[1], WCS_Info.bounds[2], WCS_Info.bounds[3])
            arr = reshape_as_image(ds.read(window))   # read all raster values and return them as (x,y,z) 


In [ ]:
 bounds = [1603849, 1637294, 1950179, 1973346], 